In [18]:
import random

from qwrapper.obs import PauliObservable
from qwrapper.hamiltonian import HeisenbergModel, compute_ground_state
from qwrapper.optimizer import AdamOptimizer, UnitLRScheduler, PrintMonitor, FileMonitor
from qswift.initializer import XBasisInitializer
from gqe.simple_model.model import SimpleModel, Ansatz
from gqe.energy_estimator.iid import IIDEstimator

In [19]:
# The file path, where the model is saved
OUTPUT_FILENAME = '../saved_models/model_three.json'

In [20]:
N = 8000
n_sample = 1000
lam = 15
nqubit = 3

# Hamiltonian and its theoretical ground state energy

In [21]:
hamiltonian = HeisenbergModel(nqubit)
print(compute_ground_state(hamiltonian))

-3.0000000000000004


# Initialize the operator pool
We define the set of operators $\{O_j\}$. Then, each gate has the form $e^{i\theta_j O_j}$. The parameters are initialized by the gaussian distribution.

In [22]:
ansatz = Ansatz([random.gauss(0, 0.5) for _ in range(18)],
                [
                    PauliObservable("XII"), PauliObservable("YII"), PauliObservable("ZII"),
                    PauliObservable("IXI"), PauliObservable("IYI"), PauliObservable("IZI"),
                    PauliObservable("IIX"), PauliObservable("IIY"), PauliObservable("IIZ"),
                    PauliObservable("XXI"), PauliObservable("YYI"), PauliObservable("ZZI"),
                    PauliObservable("IXX"), PauliObservable("IYY"), PauliObservable("IZZ"),
                    PauliObservable("XIX"), PauliObservable("YIY"), PauliObservable("ZIZ"),
                ], nqubit=nqubit)

# Initialize Energy Estimator
- The module calculates the expectation value of the Hamiltonian and its gradient for each parameter, assuming that the gate at each position is generated from an i.i.d. distribution..
- XBasisInitializer determines the initial quantum state before applying the generated quantum gates.

In [23]:
estimator = IIDEstimator(hamiltonian,
                         XBasisInitializer(),
                         N, K=0, tool='qulacs', n_sample=n_sample, n_grad_sample=1)

# Perform simple Model optimization
The model generate the sequence of gates, calculate the gradient, and update the parameters.

In [24]:
model = SimpleModel(estimator, ansatz, N, lam, n_sample)
monitors = [PrintMonitor(), FileMonitor('../output/energy.txt')]
model.run(AdamOptimizer(maxiter=100, scheduler=UnitLRScheduler(0.01), monitors=monitors))
for m in monitors:
    m.finalize()
with open(OUTPUT_FILENAME, 'w') as f:
    f.write(model.ansatz.toJSON())

0 -1.2564280749310954
1 -1.3874472650429124
2 -1.5012308593418489
3 -1.6114054000765003
4 -1.7189128848734576
5 -1.8166824983959553
6 -1.9070601017330802
7 -1.9946352440392798
8 -2.0790125955542487
9 -2.1576404939612175
10 -2.2290659650806797
11 -2.2982799349064678
12 -2.3633186688399266
13 -2.4237700147792935
14 -2.480002752755099
15 -2.5332974218325326
16 -2.5798724970505713
17 -2.620400239086182
18 -2.6554751275826622
19 -2.687069175782568
20 -2.7126519071164674
21 -2.737255566714644
22 -2.7592735274609366
23 -2.777454217194399
24 -2.793775001349224
25 -2.8082492706086857
26 -2.8212162411260966
27 -2.8331181431286767
28 -2.8449984160811206
29 -2.8564076247578343
30 -2.8671285540355917
31 -2.878012514509415
32 -2.8881757181487053
33 -2.897390036657804
34 -2.9055253213279335
35 -2.913490278061447
36 -2.9207001904784855
37 -2.9277036752210304
38 -2.9340417084358887
39 -2.939942888437273
40 -2.9454250749584165
41 -2.9502481021763125
42 -2.954926155784297
43 -2.959661588095085
44 -2.9637